# 第9章: ベクトル空間法 (I)

enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

In [1]:
import numpy as np

# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
+ 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [2]:
# head -n28443 enwiki-20150112-400-r100-10576.txt > enwiki.txt
with open('enwiki.txt', 'r') as f:
    delete_words = ".,!?;:()[]'" + '"'
    corpus = []
    for line in f.readlines():
        for token in line[:-1].split():
            if token[0] in delete_words:
                token = token[1:]
            if len(token) == 0:
                break
            if token[-1] in delete_words:
                token = token[:-1]
            if token:
                corpus.append(token)
with open('corpus.txt', 'w') as f:
    f.write(" ".join(corpus))

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [3]:
# http://www.projectvisa.com/fullcountrylist.asp
with open('country.txt', 'r') as f:
    countries = [x[:-1] for x in f.readlines() if " " in x]

with open('corpus.txt', 'r') as f:
    corpus = f.read()[:-1]

In [4]:
for c in countries:
    corpus = corpus.replace(c, c.replace(" ", "_"))

In [5]:
corpus_list = corpus.split(" ")
dictionary = np.array(list(set(corpus_list)))
T = len(corpus_list)

In [41]:
with open('dictionary.txt', 'w') as f:
    for d in dictionary:
        f.write(d + '\n')

# 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

+ ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
+ 単語$t$を選ぶ度に，文脈幅$d$は$\{1,2,3,4,5\}$の範囲でランダムに決める．

In [6]:
def get_d():
    d = np.random.randint(-5, 5)
    if d == 0:
        d = get_d()
    return d

with open("82.txt", "w") as f:
    for i, t in enumerate(corpus_list):
        d = get_d() 
        if (i + d >= 0) and  (i + d < T):
            c = corpus_list[i + d]
            if c != t:
                line = t + "\t" + c + "\n"
                f.write(line)

# 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

+ $f(t,c)$: 単語$t$と文脈語$c$の共起回数
+ $f(t,∗)$: 単語$t$の出現回数
+ $f(∗,c)$: 文脈語$c$の出現回数
+ $N$: 単語と文脈語のペアの総出現回数

In [7]:
import pandas as pd
tc = pd.read_csv("82.txt", sep="\t", header=None, names=["t", "c"])
tc = tc[tc.t != tc.c]

In [8]:
N = len(tc.index)
N

1283887

In [9]:
tc.head()

,t,c
0,Anarchism,is
1,is,philosophy
2,political,that
3,philosophy,stateless
4,that,philosophy


In [10]:
f_t = tc.groupby('t').count()
f_c = tc.groupby('c').count()
tc['count'] = tc['t']
f_tc = tc.groupby(list("tc")).count()

In [11]:
f_t.ix['a']

c    24822
Name: a, dtype: int64

In [12]:
f_c.ix['a']

t    24732
Name: a, dtype: int64

In [13]:
f_tc.ix[('a', 'that')][0]

211

# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

+ $f(t,c)≥10$ならば，$X_{tc}=PPMI(t,c)=max\{log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
+ $f(t,c)<10$ならば，$X_{tc}=0$

ここで，$PPMI(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．
なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [14]:
f_tc[f_tc['count'] >= 10].head()

count
t c         
& the     16
1 and     12
  is      10
  of      11
  on      17

In [15]:
len(f_tc[f_tc['count'] >= 10])

8652

In [16]:
from scipy import sparse
from math import log
W = len(dictionary)
X = sparse.lil_matrix((W, W))

In [17]:
dictionary[100]

'spanning'

In [18]:
np.where(dictionary == 'is')[0][0]

57069

In [19]:
dictionary[26197]

'Punjab'

In [20]:
X_df = f_tc[f_tc['count'] >= 10].reset_index()
X_df.head()

,t,c,count
0,&,the,16
1,1,and,12
2,1,is,10
3,1,of,11
4,1,on,17


In [21]:
f_t = f_t.reset_index()
f_t.columns = [['t', 'ft']]

In [22]:
f_c = f_c.reset_index()
f_c.columns = [['c', 'fc']]

In [23]:
X_df = pd.merge(X_df, f_t, on='t')
X_df = pd.merge(X_df, f_c, on='c')
X_df.head()

,t,c,count,ft,fc
0,&,the,16,228,78589
1,1,the,14,435,78589
2,10,the,12,273,78589
3,15,the,12,204,78589
4,18,the,87,1521,78589


In [24]:
X_df['ppmi1'] = (N * X_df['count']) / (X_df['ft'] * X_df['fc'])

In [25]:
X_df.head()

,t,c,count,ft,fc,ppmi1
0,&,the,16,228,78589,1.146437
1,1,the,14,435,78589,0.525780
2,10,the,12,273,78589,0.718098
3,15,the,12,204,78589,0.960984
4,18,the,87,1521,78589,0.934448


In [26]:
len(X_df[X_df['ppmi1'] > 1])

6509

In [27]:
with open('X.txt', 'w') as f:
    line = "t_i,c_i,ppmi\n"
    f.write(line)
    for row in X_df[X_df['ppmi1'] > 1].iterrows():
        t = row[1]['t']
        c = row[1]['c']
        t_i = np.where(dictionary == t)[0][0]
        c_i = np.where(dictionary == c)[0][0]
        ppmi = log(row[1]['ppmi1'])
        line = str(t_i) + "," + str(c_i) + "," + str(ppmi) + "\n"
        f.write(line)

In [28]:
X_csv = pd.read_csv('X.txt')
for row in X_csv.iterrows():
    X[row[1]['t_i'], row[1]['c_i']] = row[1]['ppmi']

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [29]:
# https://stackoverflow.com/questions/33603787/performing-pca-on-large-sparse-matrix-by-using-sklearn
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(300)
Xpca = clf.fit_transform(X)

In [30]:
with open('Xpca.csv', 'w') as f:
    for l in Xpca:
        f.write(','.join([str(x) for x in l]) + '\n')

# 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [31]:
def vec(word):
    i = np.where(dictionary == word)[0][0]
    return Xpca[i].reshape(1, -1)

In [32]:
vec("United_States")

array([[  6.32938283e-01,  -6.45292654e-01,  -9.04935904e-01,
          1.01124132e-01,  -1.07691932e-02,   7.49554064e-02,
          2.94165872e+00,  -1.19563705e+00,   1.12222834e+00,
         -2.87189609e-01,  -5.67882586e-02,   1.63305881e+00,
          3.40468329e+00,   7.15544361e-01,  -4.78274548e-01,
          1.48634771e+00,  -1.28069888e+00,  -9.64026503e-01,
         -1.55520316e+00,   5.18257039e-01,  -4.79774766e-01,
         -9.56146322e-01,  -6.64634096e-01,   8.83612013e-01,
         -4.20902763e-02,   3.17859859e-01,   2.24815016e-02,
         -9.32178743e-01,   1.09616342e-01,  -6.85292700e-01,
          2.08469988e-01,   5.34492361e-02,  -5.59082232e-01,
          2.58759918e-01,  -4.76658805e-01,  -1.13194370e+00,
          8.76637409e-01,  -1.12303905e-01,   1.80459955e-01,
          6.81766493e-02,  -3.86885282e-01,   3.72921334e-01,
          3.51680160e-01,   7.55628299e-01,   1.18997119e+00,
         -5.55607407e-01,   1.23157874e+00,   1.87059588e-01,
        

# 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
def get_word_cs(word1, word2):
    return cosine_similarity(vec(word1), vec(word2))[0][0]
get_word_cs("United_States", "U.S")

0.018007739417234062

# 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [34]:
def get_top10(word):
    lst=[]
    for i, w in enumerate(Xpca[:1000]):
        cs = cosine_similarity(vec(word), w.reshape(1, -1))[0][0]
        lst.append((dictionary[i], cs))
    return pd.DataFrame(lst)

In [35]:
eng_top10 = get_top10("England")

In [36]:
eng_top10.sort_values(1, ascending=False).head(10)

,0,1
47,events,0.806443
616,areas,0.752089
736,without,0.476834
207,3.84%,0.143077
63,seceded,0.115013
292,propitious,0.096278
177,Mosson,0.094246
301,own,0.089700
127,PEN/Laura,0.077884
174,Town.,0.067260


# 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ

In [37]:
vec89 = vec("Spain") - vec("Madrid") + vec("Athens")
lst=[]
for i, w in enumerate(Xpca[:1000]):
    cs = cosine_similarity(vec89, w.reshape(1, -1))
    lst.append((dictionary[i], cs[0][0]))
df89 = pd.DataFrame(lst)

In [38]:
df89.sort_values(1, ascending=False).head(10)

,0,1
12,Valley,0.719099
115,growth,0.489542
736,without,0.143890
16,Distal-less,0.080978
256,Isabey,0.078040
27,pears,0.071105
203,29.00%,0.071071
306,operatus,0.068993
14,"1,415",0.062753
277,Felleng,0.061202
